# 1. Lecture CSV

In [94]:
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

In [95]:
data_train = pd.read_csv("../datas/conversion_data_train.csv")
data_train.head()

,country,age,new_user,source,total_pages_visited,converted
0,China,22,1,Direct,2,0
1,UK,21,1,Ads,3,0
2,Germany,20,0,Seo,14,1
3,US,23,1,Seo,3,0
4,US,28,1,Direct,3,0


# 2. EDA

In [96]:
# Basic stats
print("Nombre de lignes : {}".format(data_train.shape[0]))
print()

print("Affichage du dataset : ")
display(data_train.head())
print()

print("Statistiques basiques : ")
data_desc = data_train.describe(include="all")
display(data_desc)
print()

print("Valeurs manquantes : ")
display(data_train.isna().sum())

print("Pourcentage de valeurs manquantes : ")
display(100 * data_train.isnull().sum() / data_train.shape[0])

Nombre de lignes : 284580

Affichage du dataset : 


,country,age,new_user,source,total_pages_visited,converted
0,China,22,1,Direct,2,0
1,UK,21,1,Ads,3,0
2,Germany,20,0,Seo,14,1
3,US,23,1,Seo,3,0
4,US,28,1,Direct,3,0



Statistiques basiques : 


,country,age,new_user,source,total_pages_visited,converted
count,284580,284580.000000,284580.000000,284580,284580.000000,284580.000000
unique,4,NaN,NaN,3,NaN,NaN
top,US,NaN,NaN,Seo,NaN,NaN
freq,160124,NaN,NaN,139477,NaN,NaN
mean,NaN,30.564203,0.685452,NaN,4.873252,0.032258
std,NaN,8.266789,0.464336,NaN,3.341995,0.176685
min,NaN,17.000000,0.000000,NaN,1.000000,0.000000
25%,NaN,24.000000,0.000000,NaN,2.000000,0.000000
50%,NaN,30.000000,1.000000,NaN,4.000000,0.000000
75%,NaN,36.000000,1.000000,NaN,7.000000,0.000000



Valeurs manquantes : 


country                0
age                    0
new_user               0
source                 0
total_pages_visited    0
converted              0
dtype: int64

Pourcentage de valeurs manquantes : 


country                0.0
age                    0.0
new_user               0.0
source                 0.0
total_pages_visited    0.0
converted              0.0
dtype: float64

In [97]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284580 entries, 0 to 284579
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   country              284580 non-null  object
 1   age                  284580 non-null  int64 
 2   new_user             284580 non-null  int64 
 3   source               284580 non-null  object
 4   total_pages_visited  284580 non-null  int64 
 5   converted            284580 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 13.0+ MB


On n'observe pas de valeurs manquantes

---

### Variable cible --> `converted`

In [98]:
data_train["converted"].value_counts()

converted
0    275400
1      9180
Name: count, dtype: int64

In [99]:
data_train["converted"].isna().sum()

np.int64(0)

La variable cible "converted" est très déséquilibrée : 275400 "0" (96.8%) et 9180 "1" (3.2%)

---

### Gestion des valeurs extrêmes

J'applique les outliers sur les différents fichiers : 
- conversion_data_train.csv
- conversion_data_test.csv
- conversion_data_test_labels.csv

In [100]:
data_test_without_labels = pd.read_csv("../datas/conversion_data_test.csv")
print("Datas test sans labels :", data_test_without_labels.shape)

labels_test = pd.read_csv("../datas/conversion_data_test_labels.csv")
print("Labels test :", labels_test.shape)

Datas test sans labels : (31620, 5)
Labels test : (31620, 1)


In [101]:
col_outliers = ["age"]
for col in col_outliers:
    # Calcul des outliers pour les colonnes concernées
    valeur_palier_haut = data_train[col].mean() + 3 * data_train[col].std()
    valeur_palier_bas = data_train[col].mean() - 3 * data_train[col].std()

    outlier_condition = (data_train[col] > valeur_palier_haut) | (data_train[col] < valeur_palier_bas)
    data_train = data_train[~outlier_condition]

    index_to_delete = data_test_without_labels[outlier_condition].index
    data_test_without_labels = data_test_without_labels.drop(index_to_delete).reset_index(drop=True)
    labels_test = labels_test.drop(index_to_delete).reset_index(drop=True)

print("Datas train :", data_train.shape)
print("Datas test sans labels :", data_test_without_labels.shape)
print("Labels test :", labels_test.shape)

Datas train : (283563, 6)
Datas test sans labels : (31514, 5)
Labels test : (31514, 1)


/tmp/ipykernel_25412/2946594377.py:10: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



---

### Types des variables

In [102]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 283563 entries, 0 to 284579
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   country              283563 non-null  object
 1   age                  283563 non-null  int64 
 2   new_user             283563 non-null  int64 
 3   source               283563 non-null  object
 4   total_pages_visited  283563 non-null  int64 
 5   converted            283563 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 15.1+ MB


#### `new_user`

In [103]:
data_train["new_user"].value_counts()

new_user
1    194366
0     89197
Name: count, dtype: int64

new_user est une variable qualitative nominale, il n'y a pas d'ordre entre les valeurs

In [104]:
data_train["new_user"] = data_train["new_user"].astype(str)
data_test_without_labels["new_user"] = data_test_without_labels["new_user"].astype(str)

`converted`

In [105]:
data_train["converted"].value_counts()

converted
0    274390
1      9173
Name: count, dtype: int64

converted est la variable cible, nous allons donc la laisser en int

In [106]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 283563 entries, 0 to 284579
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   country              283563 non-null  object
 1   age                  283563 non-null  int64 
 2   new_user             283563 non-null  object
 3   source               283563 non-null  object
 4   total_pages_visited  283563 non-null  int64 
 5   converted            283563 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 15.1+ MB


In [107]:
data_test_without_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31514 entries, 0 to 31513
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   country              31514 non-null  object
 1   age                  31514 non-null  int64 
 2   new_user             31514 non-null  object
 3   source               31514 non-null  object
 4   total_pages_visited  31514 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 1.2+ MB


---

Le dataset est important (283563 lignes), j'en crée un échantillon pour les visualisations

In [108]:
data_train_sample = data_train.sample(10000)
data_train_sample.shape

(10000, 6)

---

### Distribution des variables

##### Distribution des variables numériques

In [109]:
num_features = data_train_sample.select_dtypes(include=["float", "int"]).columns

for feature in num_features:
    fig = px.histogram(data_train_sample, x=feature, color_discrete_sequence=px.colors.qualitative.Pastel)
    fig.update_layout(title=f"Distribution de {feature}", title_x=0.5, xaxis_title=feature, yaxis_title="Nombre d'occurences")
    fig.show()

##### Répartition des variables qualitatives

In [110]:
cat_features = data_train_sample.select_dtypes(include=["object", "bool"]).columns

for feature in cat_features:
    df_grouped = data_train_sample[feature].value_counts().reset_index()
    df_grouped.columns = [feature, "count"]

    fig = px.bar(df_grouped, x=feature, y="count", color_discrete_sequence=px.colors.qualitative.Pastel)
    fig.update_layout(title=f"Répartition de {feature}", title_x=0.5,
        xaxis_title=feature, yaxis_title="Nombre d'occurences")
    fig.show()

---

### Distribution des variables en fonction de la variable cible

#### Distribution des variables numériques

In [111]:
num_features = data_train_sample.select_dtypes(include=["float", "int"]).columns

for feature in num_features:
    fig = px.histogram(data_train_sample, x=feature, color="converted", facet_row="converted", color_discrete_sequence=px.colors.qualitative.Pastel)
    fig.update_layout(title=f"Distribution de {feature}", title_x=0.5, xaxis_title=feature)
    fig.for_each_yaxis(lambda yaxis: yaxis.update(title="Nombre d'occurrences"))
    fig.for_each_annotation(lambda a: a.update(text=""))

    fig.show()

#### Répartition des variables qualitatives

In [112]:
cat_features = data_train_sample.select_dtypes(include=["object", "bool"]).columns

for feature in cat_features:
    df_grouped = data_train_sample.groupby([feature, "converted"]).size().reset_index(name="count")
    df_grouped["converted"] = df_grouped["converted"].astype(str)

    fig = px.bar(df_grouped, x=feature, y="count", color="converted", facet_row="converted", color_discrete_sequence=px.colors.qualitative.Pastel)
    fig.update_layout(title=f"Répartition de {feature}", title_x=0.5, xaxis_title=feature)
    fig.for_each_yaxis(lambda yaxis: yaxis.update(title="Nombre d'occurrences"))
    fig.for_each_annotation(lambda a: a.update(text=""))
    fig.show()

---

### Gaphique bivarié afin d'analyser la relation de chaque variable avec la cible

In [113]:
fig = px.scatter_matrix(data_train_sample)
fig.update_layout(title = go.layout.Title(text = "Bivariate analysis", x = 0.5), showlegend = False, autosize=False, height=1200, width = 1200)

fig.show()

Visuellement, je ne vois pas de corrélation évidente entre une variable du datset et la variable cible. Je vais regarder la matrice de corrélation.

---

### Matrice de corrélation

In [114]:
corr_matrix = data_train_sample.select_dtypes(include=["float", "int"]).corr().round(2)

fig = ff.create_annotated_heatmap(corr_matrix.values, x = corr_matrix.columns.tolist(), y = corr_matrix.index.tolist())
fig.show()

Grâce à la matrice de correlation, je vois que la variable `total_pages_visited` est fortement correlée avec la variable cible, je vais m'appuyer dessus pour créer ma baseline

---

Sauvegarde des fichiers après traitement

In [115]:
data_train.to_csv("../datas/conversion_data_train_EDA.csv", index=False)
data_test_without_labels.to_csv("../datas/conversion_data_test_EDA.csv", index=False)
labels_test.to_csv("../datas/conversion_data_test_labels_EDA.csv", index=False)